# Bases des télécommunications 1

Étudiants: 

Nom du groupe :

## Introduction

Le but de ce laboratoire est de mettre en pratique les concepts vus en classe de Bases des
Télécommunications. On travaillera ainsi sur la détection d'erreur sur un fichier texte.

- Le rendu du **jupyter notebook** doit se faire sur __cyberlearn__ sous le nom de fichier "prenom_nom_groupe" 
- Le fichier texte utilisé ici doit être encodé en UTF-8 (sans BOM), vous pouvez le vérifier avec Notepad++ par exemple.
- Exemple de code python 3 pour ouvrir en lecture un fichier texte en UTF-8 : __*open(INPUT, "r", encoding="UTF-8")*__

## I. Parité simple

a) Réalisez ci-dessous une **fonction** qui insère le bit de parité dans un charactère ascii. La fonction doit pouvoir calculer la parité pair ou impair

<table style=" margin-left: auto;margin-right: auto;">
    <tr>
        <td>2<sup>7</sup></td>
        <td>2<sup>6</sup></td>
        <td>2<sup>5</sup></td>
        <td>2<sup>4</sup></td>
        <td>2<sup>3</sup></td>
        <td>2<sup>2</sup></td>
        <td>2<sup>1</sup></td>
        <td>2<sup>0</sup></td>
    </tr>
    <tr>
        <td>1</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
        <td>1</td>
        <td>0</td>
        <td>1</td>
    </tr>        
    <tr>
        <td colspan="8" style=" text-align: right;">↑ <br/> bit de parité</td>
    </tr>        
</table>

Voici deux exemples qui vont permettre de comprendre comment fonctionne la création du bit de parité pair. 

**Indice: Sur combien de bit, un charactère ascii est-il codé  ?**
<table style=" margin-left: auto;margin-right: auto;">
    <tr>
    <td colspan="8" style="text-align:center" >charactère '*'</td>
    </tr>
    <tr>
        <td>2<sup>7</sup></td>
        <td>2<sup>6</sup></td>
        <td>2<sup>5</sup></td>
        <td>2<sup>4</sup></td>
        <td>2<sup>3</sup></td>
        <td>2<sup>2</sup></td>
        <td>2<sup>1</sup></td>
        <td>2<sup>0</sup></td>
    </tr>
    <tr>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>1</td>
    </tr>        
    <tr>
        <td colspan="8" style=" text-align: right;">↑ <br/> bit de parité</td>
    </tr>        
</table>

</br>
<table style=" margin-left: auto;margin-right: auto;">
    <tr>
    <td colspan="8" style="text-align:center" >charactère '('</td>
    </tr>
    <tr>
        <td>2<sup>7</sup></td>
        <td>2<sup>6</sup></td>
        <td>2<sup>5</sup></td>
        <td>2<sup>4</sup></td>
        <td>2<sup>3</sup></td>
        <td>2<sup>2</sup></td>
        <td>2<sup>1</sup></td>
        <td>2<sup>0</sup></td>
    </tr>
    <tr>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
    </tr>        
    <tr>
        <td colspan="8" style=" text-align: right;">↑ <br/> bit de parité</td>
    </tr>        
</table>

In [19]:
def generate_parity_bit(char, even=0):
    binary = format(char, '07b')
    result = 0
    parity = 0

    for i in range(0, len(binary)):
        result += int(binary[i])

    if even:
        if (result % 2 == 1):
            parity = 0
        else:
            parity = 1
    else:
        if (result % 2 == 0):
            parity = 0
        else:
            parity = 1
            
    return parity

b) Réalisez ci-dessous un programme qui affiche le nombre total d'erreur dans le fichier text (**input_with_error.bin**) à l'aide du bit de parité.

In [26]:
content = ""
nbrErrors = 0

with open("input_with_errors.bin","rb") as f:
    content = f.read()

for i, b in enumerate(content):

    b_parity = b & 0x01
    us_parity = generate_parity_bit(b >> 1)

    if (b_parity != us_parity):
        nbrErrors += 1
print(nbrErrors)

42123


c) Réaliser ci-dessous une fonction permettant:
 
 1. de calculer le bit de parité du charactère
 2. d'altérer le charactère _**dans toutes les positions possibles selon la table ASCII**_ (Conseil: utilisé la fonction **[randrange()](https://docs.python.org/3/library/random.html#random.randrange)** )
 3. ajouter le bit de parité au charactère altéré.

In [21]:
import random
def edit_char(char):
    parity = generate_parity_bit(char)
    binary = format(char, '08b')

    modified_char = ''

    randomIdx = random.randrange(0, len(binary), 1)

    modified_char = '1' if binary[randomIdx] == '0' else '0'

    modified_binary = binary[:randomIdx] + modified_char + binary[randomIdx + 1:] + str(parity)
    print(modified_binary)
    return modified_binary
    

d) Réaliser ci-dessous un programme permettant d'ajouter le bit de parité selon la machine à états suivant au fichier **"input.txt"** selon le schéma suivant et sauvegarder le résultat **"output.bin"**:

![state_machine](state_machine.jpg)

La **graine** utilisé pour créer le fichier **input_with_error.bin** est le numéro **43**.

In [22]:
import random
SEED = 43
STATE_ERROR = False
AVERAGE = 1 - (1e-6)
random.seed(43)

In [23]:
file_content = ""
with open("input.txt","rb") as f:
    file_content = f.read()

In [24]:
nb_error = 0
result = bytearray()
to_return = ""

def generate_errors(char, state_error):
    """
    Generate an error randomly. The state is state_error
    """
    global nb_error # this tells python to use the variable from the global scope
    global to_return
    # Complete the function from here
    rdm = random.random()

    if (rdm > AVERAGE):
        state_error = False if state_error else True

    print(state_error)
    if (state_error):
        to_return = edit_char(char)
        nb_error += 1
    return int(to_return, 2)

# Write the bytes in the file
for byte in file_content:
    erroneous_byte = generate_errors(byte, STATE_ERROR)
    print(erroneous_byte)
    result.append(erroneous_byte)

False


ValueError: invalid literal for int() with base 2: ''

In [58]:
print(f"Number of errors : {nb_error}")

with open("output.bin", "wb") as f:
    f.write(result)
    pass

Number of errors : 0


**⚠️ ATTENTION ⚠️** veillez à reéxecuter le code de _seeding_ lorsque vous utilisez la génération aléatoire avec la graine. Sinon vous aurez des valeurs différentes que le fichier encodé par nos soins. 

## II. Correction d'erreurs

Vérifier de deux manières différentes que votre fichier généré est identique au fichier **check_input_with_errors.bin**

a) Méthode 1

In [45]:
# Code for verification method 1 goes here
import hashlib

hashErrorAdded = hashlib.md5(open('input_with_errors_generated.bin','rb').read()).hexdigest()
hashDefaultInput = hashlib.md5(open('check_input_with_error.bin','rb').read()).hexdigest()

if hashErrorAdded == hashDefaultInput:
    print("C'est les mêmes")
print(hashErrorAdded, hashDefaultInput)

C'est les mêmes
ae9f6a902df7d45ab245dcc96e91c063 ae9f6a902df7d45ab245dcc96e91c063


b) Méthode 2

In [25]:
# Code for verification method 2 goes here
# & bit a bit

c) Actuellement le code génère beaucoup d'erreurs, comment peut-on les diminuer ?

< Réponse ici >